In [1]:
import selenium
print(selenium.__file__)

d:\Python\Lib\site-packages\selenium\__init__.py


In [2]:
import os
current_directory = os.getcwd()
print("current path of this project:", current_directory)

Đường dẫn hiện tại của project: c:\Users\MPT\Desktop\Python


In [6]:
pip install selenium

  Using cached selenium-4.24.0-py3-none-any.whl.metadata (7.1 kB)
Using cached selenium-4.24.0-py3-none-any.whl (9.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install webdriver-manager

In [1]:
import openpyxl
from openpyxl import Workbook, load_workbook
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time

In [3]:
def get_value_excel(sheet, cellname):
    return sheet[cellname].value

def update_value_excel(sheet, cellname, value):
    sheet[cellname].value = value

col_name_mst = "A"
col_name_cty = "B"
col_name_linhvuc = "C"
col_name_loai = "D"
col_name_ngdd = "E"
col_name_sdt = "F"
col_name_tinhtrang = "G"
col_name_diachi = "H"
col_name_link ="I"
filename = 'crawl-data.xlsx'

# Cấu hình WebDriver
options = Options()

# Open the Excel file once and keep it open for updating
wb = openpyxl.load_workbook(filename)
sheet = wb['Sheet3']

# Khởi tạo WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # Xác định phạm vi dòng để kiểm tra giá trị trống trong cột "D"
    max_row = sheet.max_row

    for i_row in range(2, 7):  # Assuming first row is header
        cell_name_mst = f"{col_name_mst}{i_row}"
        cell_name_cty = f"{col_name_cty}{i_row}"
        cell_name_linhvuc = f"{col_name_linhvuc}{i_row}"
        cell_name_loai = f"{col_name_loai}{i_row}"
        cell_name_ngdd = f"{col_name_ngdd}{i_row}"
        cell_name_sdt = f"{col_name_sdt}{i_row}"
        cell_name_tinhtrang = f"{col_name_tinhtrang}{i_row}"
        cell_name_diachi = f"{col_name_diachi}{i_row}"
        cell_name_link = f"{col_name_link}{i_row}"
        
        sdt_value = get_value_excel(sheet, cell_name_sdt)

        if sdt_value is None or sdt_value.strip() == "":  # Check if cell is empty
            mst = get_value_excel(sheet, cell_name_mst)

            driver.get("https://masothue.com/")

            # Tìm kiếm MST
            try:
                mstSearch = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "search"))
                )
                mstSearch.send_keys(mst)

                # Click the search button
                for attempt in range(3):  # Retry up to 3 times
                    try:
                        mstSearchBtn = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-secondary.btn-search-submit"))
                        )
                        mstSearchBtn.click()
                        break  # Exit loop if click is successful
                    except StaleElementReferenceException:
                        # Re-locate the button if it becomes stale
                        continue

                # Trích xuất dữ liệu
                mstSearchsdt = None
                mstSearchngdd = None
                mstSearchtinhtrang = None
                mstSearchlinhvuc = None
                mstSearchcompany = None
                mstSearchdiachi = None
                mstSearchloai = None
                mstSearchlink = None 

                # Trích ra sdt
                try:
                    mstSearchsdt = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "td[itemprop='telephone']"))).text
                except TimeoutException:
                    print(f"Phone number not found for row {i_row}")

                # Trích ra ng đại diện
                try:
                    mstSearchngdd = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span[itemprop='name']>a"))).text
                except TimeoutException:
                    print(f"Representative name not found for row {i_row}")

                # Trích ra tình trạng công ty
                try:
                    mstSearchtinhtrang = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "td > a[title*='tra cứu mã số thuế công ty']"))).text
                except TimeoutException:
                    print(f"Company status not found for row {i_row}")

                # Trích ra lĩnh vực công ty
                try:
                    mstSearchlinhvuc = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.XPATH, '//table[2]/tbody/tr/td[2]/strong/a'))).text
                except TimeoutException:
                    print(f"Industry not found {i_row}")

                # Trích ra tên công ty
                try:
                    mstSearchcompany = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "th[itemprop='name'] span.copy"))).text
                except TimeoutException:
                    print(f"Company name not found for row {i_row}")

                # Trích ra loại hình công ty
                try:
                    mstSearchloai = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "td > a[title*='tra cứu mã số thuế']"))).text
                except TimeoutException:
                    print(f"Business entity not found for row {i_row}")

                # Trích ra địa chỉ 
                try:
                    mstSearchdiachi = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "td[itemprop='address'] span.copy"))).text
                except TimeoutException:
                    print(f"Address not found for row {i_row}")
               
                # Trích ra link
                try:
                    mstSearchlink = driver.current_url  # Lấy URL của trang hiện tại
                except Exception:
                    print(f"Link not found for row {i_row}")

                # Cập nhật dữ liệu vào file Excel
                if mstSearchsdt is not None:
                    update_value_excel(sheet, cell_name_sdt, mstSearchsdt)
                if mstSearchngdd is not None:
                    update_value_excel(sheet, cell_name_ngdd, mstSearchngdd)
                if mstSearchtinhtrang is not None:
                    update_value_excel(sheet, cell_name_tinhtrang, mstSearchtinhtrang)
                if mstSearchlinhvuc is not None:
                    update_value_excel(sheet, cell_name_linhvuc, mstSearchlinhvuc)  
                if mstSearchcompany is not None:
                    update_value_excel(sheet, cell_name_cty, mstSearchcompany)  
                if mstSearchdiachi is not None:
                    update_value_excel(sheet, cell_name_diachi, mstSearchdiachi)  
                if mstSearchloai is not None:
                    update_value_excel(sheet, cell_name_loai, mstSearchloai)  
                if mstSearchlink is not None:
                    update_value_excel(sheet, cell_name_link, mstSearchlink) 

            except TimeoutException as ex:
                print(f"TimeoutException occurred for row {i_row}: {ex}")
                # You may want to handle this case differently depending on your needs
                continue

finally:
    driver.quit()  # Ensure WebDriver is closed properly
    wb.save(filename)  # Save changes to the Excel file
    wb.close()  # Close the Excel workbook